## Context

This data set includes customers who have paid off their loans, who have been past due and put into collection without paying back their loan and interests, and who have paid off only after they were put in collection. The financial product is a bullet loan that customers should pay off all of their loan debt in just one time by the end of the term, instead of an installment schedule. Of course, they could pay off earlier than their pay schedule.

## Content
* Loan_id A unique loan number assigned to each loan customers

* Loan_status Whether a loan is paid off, in collection, new customer yet to payoff, or paid off after the collection efforts

* Principal Basic principal loan amount at the origination

* terms Can be weekly (7 days), biweekly, and monthly payoff schedule

* Effective_date When the loan got originated and took effects

* Due_date Since it’s one-time payoff schedule, each loan has one single due date

* Paidoff_time The actual time a customer pays off the loan

* Pastdue_days How many days a loan has been past due

* Age, education, gender A customer’s basic demographic information

### Import Libraries

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as po
po.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = 'ggplot2'

### Load and Preview Data

In [3]:
loan = pd.read_csv("Loan payments data.csv")
print(loan.shape)
loan.head()

(500, 11)


,Loan_ID,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender
0,xqd20166231,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male
1,xqd20168902,PAIDOFF,1000,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female
2,xqd20160003,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female
3,xqd20160004,PAIDOFF,1000,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male
4,xqd20160005,PAIDOFF,1000,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female


### Data Cleaning/Transformation

In [4]:
ln_data = loan.copy()

In [5]:
new_col = []
for col in ln_data.columns:
    new = col.lower().strip()
    new_col.append(new)
ln_data.columns = new_col

In [6]:
ln_data.dtypes

loan_id            object
loan_status        object
principal           int64
terms               int64
effective_date     object
due_date           object
paid_off_time      object
past_due_days     float64
age                 int64
education          object
gender             object
dtype: object

In [7]:
date_col = ['effective_date', 'due_date', 'paid_off_time']
for col in date_col:
    ln_data[col] = pd.to_datetime(ln_data[col])

In [8]:
ln_data['past_due_days'] = ln_data['past_due_days'].fillna(0).astype('int')

In [9]:
ln_data.dtypes

loan_id                   object
loan_status               object
principal                  int64
terms                      int64
effective_date    datetime64[ns]
due_date          datetime64[ns]
paid_off_time     datetime64[ns]
past_due_days              int32
age                        int64
education                 object
gender                    object
dtype: object

In [10]:
ln_data.isna().sum()

loan_id             0
loan_status         0
principal           0
terms               0
effective_date      0
due_date            0
paid_off_time     100
past_due_days       0
age                 0
education           0
gender              0
dtype: int64

In [11]:
print(min(ln_data['effective_date']))
print(max(ln_data['effective_date']))

2016-09-08 00:00:00
2016-09-14 00:00:00


In [12]:
# ln_data['month'] = ln_data['effective_date'].dt.to_period('D')
# ln_data.head()

ln_data['hourly'] = ln_data['paid_off_time'].dt.strftime('%Y-%m-%d %H:00:00')
ln_data.head()

,loan_id,loan_status,principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,gender,hourly
0,xqd20166231,PAIDOFF,1000,30,2016-09-08,2016-10-07,2016-09-14 19:31:00,0,45,High School or Below,male,2016-09-14 19:00:00
1,xqd20168902,PAIDOFF,1000,30,2016-09-08,2016-10-07,2016-10-07 09:00:00,0,50,Bechalor,female,2016-10-07 09:00:00
2,xqd20160003,PAIDOFF,1000,30,2016-09-08,2016-10-07,2016-09-25 16:58:00,0,33,Bechalor,female,2016-09-25 16:00:00
3,xqd20160004,PAIDOFF,1000,15,2016-09-08,2016-09-22,2016-09-22 20:00:00,0,27,college,male,2016-09-22 20:00:00
4,xqd20160005,PAIDOFF,1000,30,2016-09-09,2016-10-08,2016-09-23 21:36:00,0,28,college,female,2016-09-23 21:00:00


In [13]:
print(ln_data['past_due_days'].unique())
print(ln_data['loan_status'].unique())

[ 0 76 61 75 60 59 74 29 44 58 73 28  2  4 13  5  3 12 27  1 25 24 36  7
 14 23  6 38 19 52 51  9 56  8]
['PAIDOFF' 'COLLECTION' 'COLLECTION_PAIDOFF']


In [14]:
def bucket(x):
    if x > 60:
        return '61-90'
    elif x > 30:
        return '31-60'
    elif x > 0:
        return '1-30'
    else:
        return 'UTD'

In [15]:
ln_data['arrears_bucket'] = ln_data['past_due_days'].apply(lambda x: bucket(x))

In [16]:
ln_data['par0'] = ln_data.apply(lambda row: row['principal'] if row['arrears_bucket'] in ['1-30', '31-60', '61-90'] else 0, axis=1)
ln_data['par30'] = ln_data.apply(lambda row: row['principal'] if row['arrears_bucket'] in ['31-60', '61-90'] else 0, axis=1)

In [17]:
ln_data['par0%'] = ln_data.apply(lambda row: row['par0']/ln_data['par0'].sum() * 100, axis=1)
ln_data['par30%'] = ln_data.apply(lambda row: row['par30']/ln_data['par30'].sum() * 100, axis=1)


In [18]:
ln_data.dtypes

loan_id                   object
loan_status               object
principal                  int64
terms                      int64
effective_date    datetime64[ns]
due_date          datetime64[ns]
paid_off_time     datetime64[ns]
past_due_days              int32
age                        int64
education                 object
gender                    object
hourly                    object
arrears_bucket            object
par0                       int64
par30                      int64
par0%                    float64
par30%                   float64
dtype: object

In [19]:
# ln_data.to_csv('laon_modified.csv', index=False)

In [20]:
grouped  = ln_data.groupby('effective_date').agg({
    'loan_id': 'count',
    'principal': 'sum',
    'past_due_days': 'mean',
    'par0': 'sum',
    'par0%': 'sum',
    'par30': 'sum',
    'par30%': 'sum'
})
grouped

,loan_id,principal,past_due_days,par0,par0%,par30,par30%
effective_date,,,,,,,
2016-09-08,4,4000,0.000000,0,0.000000,0,0.000000
2016-09-09,15,13700,25.133333,9400,4.918891,4600,4.791667
2016-09-10,46,43200,30.934783,25800,13.500785,20200,21.041667
2016-09-11,231,219000,20.246753,106700,55.834642,64800,67.500000
2016-09-12,148,139400,4.898649,49200,25.745683,6400,6.666667
2016-09-13,23,20900,0.000000,0,0.000000,0,0.000000
2016-09-14,33,31400,0.000000,0,0.000000,0,0.000000


### EDA

In [21]:
ln_data[ln_data['loan_status'] == 'COLLECTION_PAIDOFF']

,loan_id,loan_status,principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,gender,hourly,arrears_bucket,par0,par30,par0%,par30%
400,xqd20160401,COLLECTION_PAIDOFF,1000,30,2016-09-09,2016-10-08,2016-10-10 11:45:00,2,26,college,male,2016-10-10 11:00:00,1-30,1000,0,0.523286,0.0
401,xqd20160402,COLLECTION_PAIDOFF,1000,15,2016-09-09,2016-09-23,2016-09-27 17:00:00,4,28,college,male,2016-09-27 17:00:00,1-30,1000,0,0.523286,0.0
402,xqd20320403,COLLECTION_PAIDOFF,1000,30,2016-09-09,2016-11-07,2016-11-20 14:10:00,13,39,college,male,2016-11-20 14:00:00,1-30,1000,0,0.523286,0.0
403,xqd20160404,COLLECTION_PAIDOFF,1000,15,2016-09-09,2016-09-23,2016-09-28 15:38:00,5,29,Bechalor,male,2016-09-28 15:00:00,1-30,1000,0,0.523286,0.0
404,xqd20190405,COLLECTION_PAIDOFF,800,15,2016-09-09,2016-09-23,2016-09-26 17:22:00,3,33,High School or Below,male,2016-09-26 17:00:00,1-30,800,0,0.418629,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,xqd20160496,COLLECTION_PAIDOFF,1000,30,2016-09-12,2016-10-11,2016-10-14 19:08:00,3,28,High School or Below,male,2016-10-14 19:00:00,1-30,1000,0,0.523286,0.0
496,xqd20160497,COLLECTION_PAIDOFF,1000,15,2016-09-12,2016-09-26,2016-10-10 20:02:00,14,26,High School or Below,male,2016-10-10 20:00:00,1-30,1000,0,0.523286,0.0
497,xqd20160498,COLLECTION_PAIDOFF,800,15,2016-09-12,2016-09-26,2016-09-29 11:49:00,3,30,college,male,2016-09-29 11:00:00,1-30,800,0,0.418629,0.0
498,xqd20160499,COLLECTION_PAIDOFF,1000,30,2016-09-12,2016-11-10,2016-11-11 22:40:00,1,38,college,female,2016-11-11 22:00:00,1-30,1000,0,0.523286,0.0


In [22]:
ln_data[ln_data['loan_status'] == 'COLLECTION']

,loan_id,loan_status,principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,gender,hourly,arrears_bucket,par0,par30,par0%,par30%
300,xqd20160301,COLLECTION,1000,15,2016-09-09,2016-09-23,NaT,76,29,college,male,NaN,61-90,1000,1000,0.523286,1.041667
301,xqd20160302,COLLECTION,1000,30,2016-09-09,2016-10-08,NaT,61,37,High School or Below,male,NaN,61-90,1000,1000,0.523286,1.041667
302,xqd20160303,COLLECTION,1000,30,2016-09-09,2016-10-08,NaT,61,33,High School or Below,male,NaN,61-90,1000,1000,0.523286,1.041667
303,xqd20160304,COLLECTION,800,15,2016-09-09,2016-09-23,NaT,76,27,college,male,NaN,61-90,800,800,0.418629,0.833333
304,xqd20160305,COLLECTION,800,15,2016-09-09,2016-09-23,NaT,76,24,Bechalor,male,NaN,61-90,800,800,0.418629,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,xqd20160396,COLLECTION,1000,30,2016-09-11,2016-10-10,NaT,59,25,High School or Below,male,NaN,31-60,1000,1000,0.523286,1.041667
396,xqd20160397,COLLECTION,1000,30,2016-09-12,2016-10-11,NaT,58,33,High School or Below,male,NaN,31-60,1000,1000,0.523286,1.041667
397,xqd20160398,COLLECTION,800,15,2016-09-12,2016-09-26,NaT,73,39,college,male,NaN,61-90,800,800,0.418629,0.833333
398,xqd20160399,COLLECTION,1000,30,2016-09-12,2016-11-10,NaT,28,28,college,male,NaN,1-30,1000,0,0.523286,0.000000


In [26]:
status_count = ln_data['loan_status'].value_counts().to_frame(name='count')
status_count

,count
PAIDOFF,300
COLLECTION,100
COLLECTION_PAIDOFF,100


In [27]:
fig = px.bar(status_count, status_count.index, 'count')